In [1]:
import pandas as pd


df = pd.read_csv('Database_Cohort_30423_COVID-19_IHU_041723.txt', sep='\t')
print(f'Il y a {df.shape[0]} patients dans la base de données.')

Il y a 30423 patients dans la base de données.


In [2]:
print(f'Il y a {df.HCQ.isnull().sum()} patients pour lesquels nous ne connaissons pas le traitement.')
df = df[df.HCQ.notnull()]

Il y a 221 patients pour lesquels nous ne connaissons pas le traitement.


In [3]:
df['TRAITE'] = ((df.HCQ + df.AZ) == 2).astype(int)
print(f'Il y a {df.TRAITE.sum()} patients traités avec HCQ+AZ, {(df.TRAITE == 0).sum()} avec un autre traitement.')

Il y a 23172 patients traités avec HCQ+AZ, 7030 avec un autre traitement.


In [4]:
print(f'Il y a {df.CANCER.isnull().sum()} patients pour lesquels nous ne connaissons pas le passif médical.')
df = df[df.CANCER.notnull()]
print(f'Il y a {df.TRAITE.sum()} patients traités avec HCQ+AZ, {(df.TRAITE == 0).sum()} avec un autre traitement.')

Il y a 14139 patients pour lesquels nous ne connaissons pas le passif médical.
Il y a 12948 patients traités avec HCQ+AZ, 3115 avec un autre traitement.


In [5]:
import tqdm

match_cols = ['AGE', 'SEX', 'PERIOD', 'OUTPATIENT',
              'VARIANT', 'VACCINATION', 'OBESITY', 'HBP', 'DIABETE', 'ASTHMA',
              'COPD', 'CANCER', 'IMMUNODEFICIENCY', 'ChronicCardiacDiseases',
              'AutoImmuneDiseases']

matched = []

ddf = df.sample(frac=1, random_state=81)

candidates = ddf[ddf.TRAITE == 1]
for idx, r in tqdm.tqdm(ddf[ddf.TRAITE == 0].iterrows()):
    lc = candidates[(candidates[match_cols] == r[match_cols]).all(axis=1)]
    if not lc.empty:
        matched.append(idx)
        matched.append(lc.index.values[0])
        candidates = candidates.drop(index=lc.index.values[0])

ddf = df.loc[matched]
print(f'Il y a {ddf.TRAITE.sum()} patients traités avec HCQ+AZ, {(ddf.TRAITE == 0).sum()} avec un autre traitement.')

3115it [00:17, 179.90it/s]

Il y a 1725 patients traités avec HCQ+AZ, 1725 avec un autre traitement.


In [6]:
print(f'Il y a {ddf[ddf.TRAITE == 0].DEATH.sum()}/1725 morts dans le groupe non traité et {ddf[ddf.TRAITE == 1].DEATH.sum()}/1725 morts dans le groupe traité')

Il y a 15/1725 morts dans le groupe non traité et 10/1725 morts dans le groupe traité


In [7]:
from scipy.stats import chisquare


res = chisquare([1710, 15], [1715, 10])
print('La probabilité que ce résultat soit dû au hasard est de {:.2f}%'.format(res.pvalue * 100))

La probabilité que ce résultat soit dû au hasard est de 11.28%
